In [7]:
import numpy as np 
import pandas as pd
import urllib.request 
import requests, io
import urllib, re, string
import datetime

In [8]:
pd.set_option('display.notebook_repr_html', True)

def _repr_latex_(self):
    return "\centering{%s}" % self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_  # monkey patch pandas DataFrame

Data Collected 1/23/2016  

https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e=  (Mint) 

Data Collected 1/25/2016  

https://urldefense.proofpoint.com/v2/url?u=https-3A__thebiogrid.org_downloads_archives_Release-2520Archive_BIOGRID-2D3.4.144_BIOGRID-2DALL-2D3.4.144.mitab.zip&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=mI2tEi0k6-2vBnUAJ8GD2-TCPVdPWzpMbje_637goZs&e=  (BioGrid)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.innatedb.com_download_interactions_innatedb-5Fppi.mitab.gz&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=M4I-F00djsSBy83R9CVNADb-OMwpiel94VVNTR-murY&e=  (InnateDB)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__dip.doe-2Dmbi.ucla.edu_dip_File.cgi-3FFN-3D2016_tab25_dip20160731.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=D7qKgclA4IbvfDjbxccL8LSG3UsPa7AU4ant8b8qXWo&e=  (DIP)

Data Collected 1/26/2016   
https://urldefense.proofpoint.com/v2/url?u=http-3A__mentha.uniroma2.it_doDownload.php-3Ffile-3Dorganisms_9606.zip&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=RMnXZik8FZh0xdEL4OZPLacO9h0pmWzSKsHvkFp3Jt4&e=  (mentha)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.ebi.ac.uk_intact_downloads&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=F7QsaA5RcDLVV_g_n7uDXIrPdh-SJk3JYZjYAE8HzO4&e=  (IntAct)  

In [9]:
def concatliketerms(df):
    
    df.sort_values(by='Protein A (gene name)', inplace=True)    
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    oldSize = df.shape
    newSize = 0
    j = 1
    while oldSize != newSize:
        df.reset_index(inplace=True)
        df.drop('index', axis=1, inplace=True)
        df.sort_values(by='Protein A (gene name)', inplace=True) 
        # concattonate like terms and remove duplicates
        i = 1
        end = len(df.index)-1
        for index in df.index:
            if i % 100 == 0:
                print('\r', 'Loop '+ str(j)+': ' +str(i)+' Out of '+ str(len(df.index)), end=' ', flush=True)
            if index in df.index.values and index != end:
                if df.ix[index, 'Protein A (gene name)'] == df.ix[index+1, 'Protein A (gene name)']:
                    if df.ix[index, 'Protein B (gene name)'] == df.ix[index+1, 'Protein B (gene name)']:
                        df.ix[index, 'PubMed ID'] = str(df.ix[index, 'PubMed ID']) +'|'+str(df.ix[index+1, 'PubMed ID'])
                        df.ix[index, 'Source databases'] = df.ix[index, 'Source databases'] +'|'+df.ix[index+1, 'Source databases']
                        df.drop(index+1, axis=0, inplace=True)
            i += 1
        j += 1
        oldSize = newSize
        newSize = df.shape

In [10]:
# column identifiers for initial data (as taken from respective metadata)
col_name = ['Unique identifier for interactor A',
      'Unique identifier for interactor B',
      'Alternative identifier for interactor A',
      'Alternative identifier for interactor B',
      'Protein A (gene name)',
      'Protein B (gene name)',
      'Interaction detection methods',
      'First author',
      'PubMed ID',
      'NCBI Taxonomy identifier for interactor A',
      'NCBI Taxonomy identifier for interactor B',
      'Interaction types',
      'Source databases', 
      'Interaction identifier(s)',
      'Confidence score']

#### Read mint data from url and creat file

In [11]:
##read in data from Mint and write to file (only needs to be proformed once)
# content=urllib.request.urlopen('https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e= ') 

# target = open('mint.tsv', 'wb')
# for line in content:
#     target.write(line)
# target.close()

## Mint Data

In [12]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mint.tsv', sep='\t', index_col=False, header=None)

FileNotFoundError: File b'/Users/moshesilverstein/Desktop/PPI Library/Input/mint.tsv' does not exist

In [ ]:
# assign names to columns from meta data
for i in np.arange(len(col_name)):
    mint_df.rename(columns={i:col_name[i]}, inplace=True)

In [ ]:
# get only human (or mouse) data
n = mint_df['Protein A (gene name)'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
mint_df = mint_df.ix[b]

In [ ]:
mint_df =  mint_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = mint_df['Protein B (gene name)'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
mint_df = mint_df.ix[b]

In [ ]:
# select only relevent data
mint_df = mint_df[['Protein A (gene name)', 'Protein B (gene name)', 'PubMed ID', 'Source databases']]

In [ ]:
# change column one to just show gene name
#i = 1
lst = []
for name in mint_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein A (gene name)'] = lst

In [ ]:
# change column two to just show gene name
#i = 1
lst = []
for name in mint_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein B (gene name)'] = lst

In [ ]:
# drop data for which no gene name is provited (or ensamble ID)
mint_df.dropna(how='any', inplace=True, axis=0)

In [ ]:
lst = []

for pub in mint_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[1].split(':')[1]
    lst.append(pub)
mint_df['PubMed ID'] = lst

In [ ]:
mint_df.drop_duplicates(inplace=True)

In [ ]:
mint_uf_df = mint_df.copy()

In [ ]:
concatliketerms(mint_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
lst = []
for pub in mint_df['PubMed ID'].values:
    print('\r', str(i)+' Out of '+ str(len(mint_df['PubMed ID'].values)), end=' ', flush=True)
    lst.append(mint_df[mint_df['PubMed ID'] == pub].shape[0] <= 10)
    i +=1

mint_df = mint_df[lst]

In [ ]:
# Drop data for which there is no PubMed ID
lst = []
for value in mint_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mint_df = mint_df.drop(mint_df[lst].index)

In [ ]:
# Drop any data missing information
mint_df.dropna(inplace=True)

In [ ]:
concatliketerms(mint_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_%s.tsv'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_%s.tsv.gz'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## BioGrid Data

In [ ]:
biogrid_df =https://urldefense.proofpoint.com/v2/url?u=http-3A__pd.re&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=kUtB02_d29l-AX62DWgGfOrrMBgeV15cPbw0hAtCi7Y&e= ad_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/BIOGRID-ALL-3.4.144.mitab.txt', sep='\t', index_col=False)

In [ ]:
# get only human (or mouse) data
n = biogrid_df['Taxid Interactor A'].values
b = [i for i,item in enumerate(n) if "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [ ]:
biogrid_df =  biogrid_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = biogrid_df['Taxid Interactor B'].values
b = [i for i,item in enumerate(n) if "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [ ]:
# select only relevent data
biogrid_df = biogrid_df[['Alt IDs Interactor A', 'Alt IDs Interactor B', 'Publication Identifiers', 'Source Database']]

In [ ]:
# rename columns for consistency
biogrid_df.rename(columns={'Alt IDs Interactor A': 'Protein A (gene name)', 'Alt IDs Interactor B':'Protein B (gene name)', 'Publication Identifiers':'PubMed ID', 'Source Database':'Source databases'}, inplace=True)

In [ ]:
# change column one to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein A (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein A (gene name)'] = lst

In [ ]:
# change column two to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein B (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein B (gene name)'] = lst

In [ ]:
# drop data for which no gene name is provited (or ensamble ID)
biogrid_df.dropna(how='any', inplace=True, axis=0)

In [ ]:
# Get Only PubMed ID for publication identifier
lst = []

for pub in biogrid_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
biogrid_df['PubMed ID'] = lst

In [ ]:
biogrid_df.drop_duplicates(inplace=True)

In [ ]:
# uf = unfiltered
biogrid_uf_df = biogrid_df.copy()

In [ ]:
concatliketerms(biogrid_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_%s.tsv'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_%s.tsv.gz'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in biogrid_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(biogrid_df['PubMed ID'].unique())), end=' ', flush=True)
    if biogrid_df[biogrid_df['PubMed ID'] == pub].shape[0] > 10:
        biogrid_df.drop(biogrid_df[biogrid_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

In [ ]:
biogrid_df.shape

In [ ]:
lst = []
for value in biogrid_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

biogrid_df = biogrid_df.drop(biogrid_df[lst].index)

In [ ]:
biogrid_df.dropna(inplace=True)

In [ ]:
concatliketerms(biogrid_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## InnateDB Data

In [ ]:
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/innatedb_ppi.mitab', sep='\t', index_col=False)

In [ ]:
# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_A'].values
b = [i for i,item in enumerate(n) if "Mouse" in item]
innatedb_df = innatedb_df.ix[b]

In [ ]:
innatedb_df =  innatedb_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_B'].values
b = [i for i,item in enumerate(n) if "Mouse" in item]
innatedb_df = innatedb_df.ix[b]

In [ ]:
innatedb_df = innatedb_df[['alias_A', 'alias_B', 'pmid', 'source_database']]

In [ ]:
innatedb_df.rename(columns={'alias_A': 'Protein A (gene name)', 'alias_B':'Protein B (gene name)', 'pmid':'PubMed ID', 'source_database':'Source databases'}, inplace=True)

In [ ]:
# change column one to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein A (gene name)'] = lst

In [ ]:
# change column two to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein B (gene name)'] = lst

In [ ]:
# drop data for which no gene name is provited (or ensamble ID)
innatedb_df.dropna(how='any', inplace=True, axis=0)

In [ ]:
lst = []

for pub in innatedb_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
innatedb_df['PubMed ID'] = lst

In [ ]:
innatedb_df.drop_duplicates(inplace=True)

In [ ]:
innatedb_uf_df = innatedb_df.copy()

In [ ]:
concatliketerms(innatedb_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [ ]:
innatedb_df.shape

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in innatedb_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(innatedb_df['PubMed ID'].unique())), end=' ', flush=True)
    if innatedb_df[innatedb_df['PubMed ID'] == pub].shape[0] > 10:
        innatedb_df.drop(innatedb_df[innatedb_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

In [ ]:
lst = []
for value in innatedb_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

innatedb_df = innatedb_df.drop(innatedb_df[lst].index)

In [ ]:
innatedb_df.dropna(inplace=True)

In [ ]:
concatliketerms(innatedb_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## DIP DATA

In [ ]:
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/dip20160731.txt', sep='\t', index_col=False)

In [ ]:
# get only human (or mouse) data
n = dip_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "Mus musculus" in item]
dip_df = dip_df.ix[b]

In [ ]:
dip_df =  dip_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = dip_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "Mus musculus" in item]
dip_df = dip_df.ix[b]

In [ ]:
dip_df = dip_df[['ID interactor A', 'ID interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [ ]:
dip_df.rename(columns={'ID interactor A': 'Protein A (gene name)', 'ID interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

#### mapping table to convert labels from uniprot to ncbi names

In [ ]:
mapping_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Doc and Mapping/MOUSE_10090_idmapping.dat', sep='\t', index_col=False, names=['UniProtKB-AC', 'ID_type', 'ID'])

In [ ]:
mapping_df = mapping_df[mapping_df['ID_type']=='Gene_Name']

In [ ]:
mapping_df.set_index('UniProtKB-AC', inplace=True)

In [ ]:
mapping_df = mapping_df[~mapping_df.index.duplicated(keep='first')]

In [ ]:
# change column one to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein A (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein A (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein A (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein A (gene name)'] = np.nan

In [ ]:
# change column two to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein B (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein B (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein B (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein B (gene name)'] = np.nan

In [ ]:
lst = []

for pub in dip_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[0].split(':')[1]
    lst.append(pub)
dip_df['PubMed ID'] = lst

In [ ]:
dip_df.drop_duplicates(inplace=True)

In [ ]:
dip_uf_df = dip_df.copy()

In [ ]:
concatliketerms(dip_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in dip_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(dip_df['PubMed ID'].unique())), end=' ', flush=True)
    if dip_df[dip_df['PubMed ID'] == pub].shape[0] > 10:
        dip_df.drop(dip_df[dip_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

In [ ]:
lst = []
for value in dip_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

dip_df = dip_df.drop(dip_df[lst].index)

In [ ]:
dip_df.dropna(inplace=True)

In [ ]:
#mapping_df.head()

In [ ]:
concatliketerms(dip_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## mentha data

In [ ]:
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mentha_mouse', sep=';')

In [ ]:
mentha_df = mentha_df[['Gene A', 'Gene B', 'PMID']]

In [ ]:
mentha_df['Source databases'] = '(mentha)'

In [ ]:
mentha_df.rename(columns={'Gene A': 'Protein A (gene name)', 'Gene B':'Protein B (gene name)', 'PMID':'PubMed ID'}, inplace=True)

In [ ]:
i = 1
for index in mentha_df.index:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df.index)), end=' ', flush=True)
    mentha_df.ix[index, 'PubMed ID'] = '|'.join(mentha_df.ix[index, 'PubMed ID'].split(' ')[0:-1])
    i +=1

In [ ]:
mentha_df.drop_duplicates(inplace=True)

In [ ]:
mentha_uf_df = mentha_df.copy()

In [ ]:
concatliketerms(mentha_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in mentha_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df['PubMed ID'].unique())), end=' ', flush=True)
    if mentha_df[mentha_df['PubMed ID'] == pub].shape[0] > 10:
        mentha_df.drop(mentha_df[mentha_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

In [ ]:
lst = []
for value in mentha_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mentha_df = mentha_df.drop(mentha_df[lst].index)

In [ ]:
mentha_df.dropna(inplace=True)

In [ ]:
mentha_df.shape

In [ ]:
concatliketerms(mentha_df)

In [ ]:
mentha_df.head()

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## IntAct Data 

In [ ]:
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/intact.txt', sep='\t')

In [ ]:
#intact_negative_df = pd.read_csv('intact_negative.txt', sep='\t')

In [ ]:
#intact_df = pd.concat([intact_df, intact_negative_df])

In [ ]:
# get only human (or mouse) data
n = intact_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
intact_df = intact_df.ix[b]

In [ ]:
intact_df =  intact_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = intact_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
intact_df = intact_df.ix[b]

In [ ]:
#intact_df.ix[0, 'Taxid interactor B']

In [ ]:
intact_df = intact_df[['Alias(es) interactor A', 'Alias(es) interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [ ]:
intact_df.rename(columns={'Alias(es) interactor A': 'Protein A (gene name)', 'Alias(es) interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

In [ ]:
# change column one to just show gene name
#i = 1
lst = []
for name in intact_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein A (gene name)'] = lst

In [ ]:
# change column two to just show gene name
#i = 1
lst = []
for name in intact_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein B (gene name)'] = lst

In [ ]:
# drop data for which no gene name is provited (or ensamble ID)
intact_df.dropna(how='any', inplace=True, axis=0)

In [ ]:
lst = []

for pub in intact_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')
    for p in pub:
        if 'pubmed' in p:
            pub = p.split(':')[1]
    lst.append(pub)
intact_df['PubMed ID'] = lst

In [ ]:
intact_df.drop_duplicates(inplace=True)

In [ ]:
intact_uf_df = intact_df.copy()

In [ ]:
concatliketerms(intact_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [ ]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in intact_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(intact_df['PubMed ID'].unique())), end=' ', flush=True)
    if intact_df[intact_df['PubMed ID'] == pub].shape[0] > 10:
        intact_df.drop(intact_df[intact_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


In [ ]:
intact_df.shape

In [ ]:
lst = []
for value in intact_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

intact_df = intact_df.drop(intact_df[lst].index)

In [ ]:
intact_df.dropna(inplace=True)

In [ ]:
intact_df.head()

In [ ]:
#intact_negative_df.head()

In [ ]:
# Alt. ID(s) interactor A
#intact_df.ix[22, 'Aliases for A']

In [ ]:
concatliketerms(intact_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False, compression='gzip')

### Data Tables

In [ ]:
mint_df.head()

In [ ]:
biogrid_df.head()

In [ ]:
innatedb_df.head()

In [ ]:
dip_df.head()

In [ ]:
mentha_df.head()

In [ ]:
intact_df.head()

## Comparison and overlap

In [ ]:
data_sets= [mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df]
data_sets_names= ['mint', 'biogrid', 'innatedb', 'dip', 'mentha', 'intact']

comp_df = pd.DataFrame(columns=data_sets_names, index=data_sets_names)
comp_df.columns.name = 'Overlap (Mouse)'
for i in np.arange(len(data_sets)):
    for j in np.arange(len(data_sets)):
        comp = pd.concat([data_sets[i][['Protein A (gene name)', 'Protein B (gene name)']], data_sets[j][['Protein A (gene name)', 'Protein B (gene name)']]])
        comp_df.ix[data_sets_names[i], data_sets_names[j]] = comp.duplicated().sum()

# mint_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
# biogrid_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
comp_df

## Merged Network Table

In [ ]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_2017_02_15.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_2017_02_15.tsv', sep='\t')

In [ ]:
network_df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df])

In [ ]:
#network_df.rename(columns={'Aliases for A':'Protein A (gene name)', 'Aliases for B':'Protein B (gene name)', 'Identifier of the publication':'PubMed ID'}, inplace=True)

In [ ]:
network_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [ ]:
network_df.reset_index(inplace=True)
network_df.drop('index', axis=1, inplace=True)

In [ ]:
network_df.shape

In [ ]:
concatliketerms(network_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [ ]:
network_df.head()

In [ ]:
mint_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_2017_02_15.tsv', sep='\t')
biogrid_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_2017_02_15.tsv', sep='\t')
innatedb_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_2017_02_15.tsv', sep='\t')
dip_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_2017_02_15.tsv', sep='\t')
mentha_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_2017_02_15.tsv', sep='\t')
intact_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_2017_02_15.tsv', sep='\t')

In [ ]:
network_uf_df = pd.concat([mint_uf_df, biogrid_uf_df, innatedb_uf_df, dip_uf_df, mentha_uf_df, intact_uf_df])

In [ ]:
network_uf_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [ ]:
network_uf_df.reset_index(inplace=True)
network_uf_df.drop('index', axis=1, inplace=True)

In [ ]:
concatliketerms(network_uf_df)

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_uf_df.to_csv(filename, sep='\t', index=False) 

In [ ]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## Merged DataFrame 

In [ ]:
# drop PPI to self for the DIP data 
for index in dip_df.index:
    if dip_df.ix[index, 'Aliases for A'] == dip_df.ix[index, 'Aliases for B']:
        dip_df.drop(index, inplace=True)

In [ ]:
df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df])

In [ ]:
df = df[['Aliases for A', 'Aliases for B']]

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(['Aliases for A', 'Aliases for B'], inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

## Gene Set Library

In [ ]:
# build library

i = 1

genelist = set(df['Aliases for A'].values.tolist() + df['Aliases for B'].values.tolist())

lib = pd.DataFrame()

for gene in genelist:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(genelist)), end=' ', flush=True)
    lst = set(df[df.values == gene].values.flatten().tolist())
    lst.remove(gene)
    lst = list(lst)
    lst.insert(0, gene)
    if len(lst) >= 50:
        temp = pd.DataFrame(data=lst)
        lib = pd.concat([lib, temp.T])
    i += 1

In [ ]:
lib.to_csv('PPI_MOUSE.gmt', sep='\t', header=None, index=False)

In [ ]:
lib.head(20)

In [ ]:
lib.shape

## Protein Count

In [ ]:
mint_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_2017_02_15.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_2017_02_15.tsv', sep='\t')

In [ ]:
len(set(mint_df['Protein A (gene name)'].tolist()+mint_df['Protein B (gene name)'].tolist()))

In [ ]:
len(set(biogrid_df['Protein A (gene name)'].tolist()+biogrid_df['Protein B (gene name)'].tolist()))

In [ ]:
len(set(innatedb_df['Protein A (gene name)'].tolist()+innatedb_df['Protein B (gene name)'].tolist()))

In [ ]:
len(set(dip_df['Protein A (gene name)'].tolist()+dip_df['Protein B (gene name)'].tolist()))

In [ ]:
len(set(mentha_df['Protein A (gene name)'].tolist()+mentha_df['Protein B (gene name)'].tolist()))

In [ ]:
len(set(intact_df['Protein A (gene name)'].tolist()+intact_df['Protein B (gene name)'].tolist()))